# Taller Práctico 3: Procesamiento de Video en Algoritmos Paralelos

## Transformando un Video a Escala de Grises

**Universidad Sergio Arboleda - Noviembre 2025**

**Juan Hurtado - Miguel Flechas - Andres Castro**
---

### Objetivo
Implementar y comparar la eficiencia de un **algoritmo secuencial** y uno **paralelo multihilos** al procesar un video, transformando el video original a escala de grises.

### INSTRUCCIONES PARA COLAB:
1. Sube tu video a Google Colab (arrastra y suelta en la carpeta de archivos)
2. Modifica la ruta del video en la celda de configuración
3. Ejecuta "Runtime > Run all"
4. Descarga el video resultante

In [15]:
# Instalación de dependencias
!pip install opencv-python numpy -q
print("✅ Dependencias instaladas")

✅ Dependencias instaladas


In [16]:
import cv2
import os
import shutil
import numpy as np
import time
from multiprocessing import Pool, cpu_count

print("✅ Librerías importadas")
print(f"   CPUs disponibles: {cpu_count()}")

✅ Librerías importadas
   CPUs disponibles: 2


In [17]:
# CONFIGURACIÓN - RUTA DEL VIDEO AQUÍ
VIDEO_PATH = "/content/rana.mp4"

FOLDER_VIDEO = "video"
FOLDER_FRAMES_ORIGINAL = "frames_video_original"
FOLDER_FRAMES_GRAY = "frames_video_gray"
VIDEO_OUTPUT_FPS = 30
VIDEO_OUTPUT_NAME = "video_escala_grises.mp4"

print("✅ Configuración establecida")

✅ Configuración establecida


In [18]:
# Preparar ambiente
for folder in [FOLDER_VIDEO, FOLDER_FRAMES_ORIGINAL, FOLDER_FRAMES_GRAY]:
    if os.path.exists(folder):
        shutil.rmtree(folder)
    os.makedirs(folder, exist_ok=True)

print("✅ Carpetas creadas")

✅ Carpetas creadas


In [19]:
# Cargar video
if not os.path.exists(VIDEO_PATH):
    raise FileNotFoundError(f"❌ No se encontró el video: {VIDEO_PATH}")

video_name = os.path.basename(VIDEO_PATH)
video_work_path = os.path.join(FOLDER_VIDEO, video_name)
shutil.copy2(VIDEO_PATH, video_work_path)

cap = cv2.VideoCapture(video_work_path)
fps_original = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
duration = total_frames / fps_original if fps_original > 0 else 0
cap.release()

print("="*70)
print("📹 INFORMACIÓN DEL VIDEO")
print("="*70)
print(f"   Archivo: {video_name}")
print(f"   Resolución: {width}x{height}")
print(f"   FPS: {fps_original:.2f}")
print(f"   Total frames: {total_frames}")
print(f"   Duración: {duration:.2f} segundos")
print("="*70)

📹 INFORMACIÓN DEL VIDEO
   Archivo: rana.mp4
   Resolución: 576x576
   FPS: 30.00
   Total frames: 140
   Duración: 4.67 segundos


In [20]:
# Extraer frames
print("📸 Extrayendo frames...\n")

cap = cv2.VideoCapture(video_work_path)
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_filename = f"frame_{frame_count:09d}.jpg"
    frame_path = os.path.join(FOLDER_FRAMES_ORIGINAL, frame_filename)
    cv2.imwrite(frame_path, frame)
    frame_count += 1

cap.release()
print(f"✅ {frame_count} frames extraídos")

📸 Extrayendo frames...

✅ 140 frames extraídos


In [21]:
# Función de conversión
def rgb_to_grayscale(rgb_image):
    return np.mean(rgb_image, axis=2).astype(np.uint8)

print("✅ Función definida")

✅ Función definida


In [22]:
# ALGORITMO SECUENCIAL
def procesar_frame_secuencial(frame_idx, folder_original, folder_gray):
    try:
        filename = f"frame_{frame_idx:09d}.jpg"
        input_path = os.path.join(folder_original, filename)
        output_path = os.path.join(folder_gray, filename)

        bgr_image = cv2.imread(input_path)
        if bgr_image is None:
            return False

        rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
        gray_image = rgb_to_grayscale(rgb_image)
        cv2.imwrite(output_path, gray_image)
        return True
    except:
        return False

def procesamiento_secuencial(total_frames, folder_original, folder_gray):
    print("="*70)
    print("ALGORITMO SECUENCIAL")
    print("="*70)

    start_time = time.time()
    processed_count = 0

    for i in range(total_frames):
        if procesar_frame_secuencial(i, folder_original, folder_gray):
            processed_count += 1

    elapsed_time = time.time() - start_time

    print(f"✅ Completado")
    print(f"   Frames: {processed_count}/{total_frames}")
    print(f"   Tiempo: {elapsed_time:.4f} segundos")
    print("="*70)

    return elapsed_time

print("✅ Algoritmo secuencial definido")

✅ Algoritmo secuencial definido


In [23]:
# ALGORITMO PARALELO
def procesar_frame_paralelo_wrapper(args):
    frame_idx, folder_original, folder_gray = args
    try:
        filename = f"frame_{frame_idx:09d}.jpg"
        input_path = os.path.join(folder_original, filename)
        output_path = os.path.join(folder_gray, filename)

        bgr_image = cv2.imread(input_path)
        if bgr_image is None:
            return None

        rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
        gray_image = np.mean(rgb_image, axis=2).astype(np.uint8)
        cv2.imwrite(output_path, gray_image)
        return frame_idx
    except:
        return None

def procesamiento_paralelo(total_frames, folder_original, folder_gray, num_cores=None):
    if num_cores is None:
        num_cores = cpu_count()

    print("="*70)
    print("ALGORITMO PARALELO MULTICORE")
    print("="*70)
    print(f"Usando {num_cores} núcleos\n")

    start_time = time.time()

    args_list = [(i, folder_original, folder_gray) for i in range(total_frames)]

    with Pool(processes=num_cores) as pool:
        results = pool.map(procesar_frame_paralelo_wrapper, args_list)

    processed_count = sum(1 for r in results if r is not None)
    elapsed_time = time.time() - start_time

    print(f"✅ Completado")
    print(f"   Frames: {processed_count}/{total_frames}")
    print(f"   Núcleos: {num_cores}")
    print(f"   Tiempo: {elapsed_time:.4f} segundos")
    print("="*70)

    return elapsed_time

print("✅ Algoritmo paralelo definido")

✅ Algoritmo paralelo definido


In [24]:
# EJECUTAR SECUENCIAL
if os.path.exists(FOLDER_FRAMES_GRAY):
    shutil.rmtree(FOLDER_FRAMES_GRAY)
os.makedirs(FOLDER_FRAMES_GRAY, exist_ok=True)

tiempo_secuencial = procesamiento_secuencial(frame_count, FOLDER_FRAMES_ORIGINAL, FOLDER_FRAMES_GRAY)

ALGORITMO SECUENCIAL
✅ Completado
   Frames: 140/140
   Tiempo: 4.0116 segundos


In [25]:
# EJECUTAR PARALELO
if os.path.exists(FOLDER_FRAMES_GRAY):
    shutil.rmtree(FOLDER_FRAMES_GRAY)
os.makedirs(FOLDER_FRAMES_GRAY, exist_ok=True)

tiempo_paralelo = procesamiento_paralelo(frame_count, FOLDER_FRAMES_ORIGINAL, FOLDER_FRAMES_GRAY)

ALGORITMO PARALELO MULTICORE
Usando 2 núcleos

✅ Completado
   Frames: 140/140
   Núcleos: 2
   Tiempo: 3.8168 segundos


In [26]:
# ANÁLISIS DE RESULTADOS
speedup = tiempo_secuencial / tiempo_paralelo
eficiencia = (speedup / cpu_count()) * 100

print("\n" + "="*70)
print("ANÁLISIS DE RESULTADOS Y SPEEDUP")
print("="*70)
print(f"\nHardware: {cpu_count()} núcleos CPU")
print(f"\nTiempos de Ejecución:")
print(f"  Secuencial:  {tiempo_secuencial:.4f} segundos")
print(f"  Paralelo:    {tiempo_paralelo:.4f} segundos")
print(f"\nSpeedup: {speedup:.2f}x")
print(f"Eficiencia: {eficiencia:.2f}%")
print(f"\nEl algoritmo paralelo es {speedup:.2f}x más rápido")
print(f"Tiempo ahorrado: {tiempo_secuencial - tiempo_paralelo:.4f} segundos")
print("="*70)


ANÁLISIS DE RESULTADOS Y SPEEDUP

Hardware: 2 núcleos CPU

Tiempos de Ejecución:
  Secuencial:  4.0116 segundos
  Paralelo:    3.8168 segundos

Speedup: 1.05x
Eficiencia: 52.55%

El algoritmo paralelo es 1.05x más rápido
Tiempo ahorrado: 0.1948 segundos


In [27]:
# GENERAR VIDEO
print("\n" + "="*70)
print("GENERANDO VIDEO EN ESCALA DE GRISES")
print("="*70)

first_frame_path = os.path.join(FOLDER_FRAMES_GRAY, "frame_000000000.jpg")
first_frame = cv2.imread(first_frame_path, cv2.IMREAD_GRAYSCALE)
height, width = first_frame.shape
frame_size = (width, height)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(VIDEO_OUTPUT_NAME, fourcc, VIDEO_OUTPUT_FPS, frame_size, isColor=False)

for i in range(frame_count):
    filename = f"frame_{i:09d}.jpg"
    frame_path = os.path.join(FOLDER_FRAMES_GRAY, filename)
    frame = cv2.imread(frame_path, cv2.IMREAD_GRAYSCALE)
    if frame is not None:
        video_writer.write(frame)

video_writer.release()

print(f"\n✅ Video generado: {VIDEO_OUTPUT_NAME}")
print("="*70)


GENERANDO VIDEO EN ESCALA DE GRISES

✅ Video generado: video_escala_grises.mp4


In [28]:
# RESUMEN FINAL
print("\n" + "="*70)
print("RESUMEN DEL TALLER")
print("="*70)
print(f"\nVideo: {video_name}")
print(f"Frames: {frame_count}")
print(f"Resolución: {width}x{height}")
print(f"\nTiempo Secuencial: {tiempo_secuencial:.4f}s")
print(f"Tiempo Paralelo: {tiempo_paralelo:.4f}s")
print(f"Speedup: {speedup:.2f}x")
print(f"Eficiencia: {eficiencia:.2f}%")
print(f"\nVideo resultante: {VIDEO_OUTPUT_NAME}")
print("\n✅ TALLER COMPLETADO")
print("="*70)


RESUMEN DEL TALLER

Video: rana.mp4
Frames: 140
Resolución: 576x576

Tiempo Secuencial: 4.0116s
Tiempo Paralelo: 3.8168s
Speedup: 1.05x
Eficiencia: 52.55%

Video resultante: video_escala_grises.mp4

✅ TALLER COMPLETADO
